# Análise do risco de inadimplência dos mutuários

Seu projeto é preparar um relatório para a divisão de empréstimos de um banco. Você precisará descobrir se o estado civil de um cliente e o número de filhos têm impacto sobre se ele deixará de pagar um empréstimo. O banco já tem alguns dados sobre a capacidade de crédito dos clientes.

Seu relatório será considerado ao criar uma **pontuação de crédito** de um cliente em potencial. A **contagem de crédito** é usada para avaliar a capacidade de um devedor em potencial de pagar seu empréstimo.

Primeiramente, os dados serão avaliados a procurá de dados ausentes, problemas nas colunas, dados com problemas, dados com o tipo errado, dados duplicados e ausentes. Depois de identificados os problemas, eles serão corrigidos para que a as hipóteses sejam testadas.

Com a tabela pronta para a ánalise serão feitas as perguntas:
- Existe alguma relação entre ter filhos e pagar um empréstimo em dia?
- Existe alguma relação entre o estado civil e o pagamento de um empréstimo no prazo estipulado?
- Existe uma relação entre o nível de renda e o pagamento de um empréstimo no prazo?
- Como as diferentes finalidades do empréstimo afetam o pagamento pontual do empréstimo?


## Abra o arquivo de dados e veja as informações gerais.


In [1]:
import pandas as pd
import numpy as np

try:
    df = pd.read_csv('credit_scoring_eng.csv')
    
except:
    df = pd.read_csv('/datasets/credit_scoring_eng.csv')

## Exploração de dados

**Descrição dos dados**
- `children` - o número de crianças na família
- `days_employed` - experiência de trabalho em dias
- `dob_years` - idade do cliente em anos
- `education` - educação do cliente
- `education_id` - identificador de educação
- `family_status` - estado civil do cliente
- `family_status_id` - identificador de estado civil
- `gender` - gênero do cliente
- `income_type` - tipo de emprego
- `debt` - havia alguma dívida no pagamento do empréstimo
- `total_income` - renda mensal
- `purpose` - o objetivo de obter um empréstimo


In [2]:
# Vamos ver quantas linhas e colunas nosso conjunto de dados tem
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [4]:
# vamos exibir as primeiras 10 linhas
df.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


A partir da primeira visualização foram encontrados alguns problemas nos dados de certas colunas: 
- A coluna **`days_employed`** possui números negativos e alguns valores em quantidade incorreta, também possui valores ausentes. 
- A coluna **`education`**  possui diferentes nomes escritos de formas diferentes.
- A coluna **`total_income`** possui valores ausentes.

In [5]:
# Obter informações sobre dados
df.describe()


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


In [6]:
df.isna().sum()\
.sort_values(ascending=False)\
.reset_index()\
.rename(columns={'index':'coluna', 0:'quantidade'})

,coluna,quantidade
0,days_employed,2174
1,total_income,2174
2,children,0
3,dob_years,0
4,education,0
5,education_id,0
6,family_status,0
7,family_status_id,0
8,gender,0
9,income_type,0


As colunas **`days_employed`** e **`total_income`** possuem ambas 2174 valores ausentes.

In [7]:
# Vejamos a tabela filtrada com valores ausentes na primeira coluna com dados ausentes
df[df['days_employed'].isna()].head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding


In [8]:
# Verificar a distribuição dos valores ausentes em days_employed e total_income em relação ao total.
df.isna().mean().sort_values(ascending=False)\
.reset_index()\
.rename(columns={'index':'coluna', 0:'porcentagem'})

,coluna,porcentagem
0,days_employed,0.100999
1,total_income,0.100999
2,children,0.000000
3,dob_years,0.000000
4,education,0.000000
5,education_id,0.000000
6,family_status,0.000000
7,family_status_id,0.000000
8,gender,0.000000
9,income_type,0.000000


**Conclusão intermediária**

Os valores nas linhas com valores ausentes parecem simétricos. Sendo a primeira vista, os valores ausentes em **`days_employed`** ocorrem nas mesmas linhas que os valores ausentes em **`total_income`**. As linhas com valores ausentes representam 2174 linhas, 10% das linhas possuem valores ausentes.

Agora será feita uma comparação entre a distribuição dos valores das colunas, entre a tabela **`df`**(possuí os valores originais) e a tabela **`df_nan`**(possuí os valores filtrados das linhas com valores ausentes). Essa comparação tem o objetivo de identificar possíveis características dos cliente que causem a ausência dos valores.

In [9]:
# Vamos investigar clientes que não possuem dados sobre as características identificadas e a coluna com os valores ausentes
df_nan= df[df['days_employed'].isna()]

# children 
print('Comparação da coluna children entre a tabela df e df_nan:')
print(df['children'].value_counts(dropna=True,normalize=True))
print(df_nan['children'].value_counts(normalize=True))
print()

# education_id days_employed 	dob_years 	education 	education_id 	family_status 	family_status_id 	gender 	income_type 	debt 	total_income 	purpose
print('Comparação da coluna education_id entre a tabela df e df_nan:')
print(df['education_id'].value_counts(dropna=True,normalize=True))
print(df_nan['education_id'].value_counts(normalize=True))
print()

# family_status_id
print('Comparação da coluna family_status_id entre a tabela df e df_nan:')
print(df['family_status_id'].value_counts(dropna=True,normalize=True))
print(df_nan['family_status_id'].value_counts(normalize=True))
print()
 
# gender
print('Comparação da coluna gender entre a tabela df e df_nan:')
print(df['gender'].value_counts(dropna=True,normalize=True))
print(df_nan['gender'].value_counts(normalize=True))
print()

# income_type
print('Comparação da coluna income_type entre a tabela df e df_nan:')
print(df['income_type'].value_counts(dropna=True,normalize=True))
print(df_nan['income_type'].value_counts(normalize=True))
print()

#debt
print('Comparação da coluna debt entre a tabela df e df_nan:')
print(df['debt'].value_counts(dropna=True,normalize=True))
print(df_nan['debt'].value_counts(normalize=True))
print()


Comparação da coluna children entre a tabela df e df_nan:
 0     0.657329
 1     0.223833
 2     0.095470
 3     0.015331
 20    0.003531
-1     0.002184
 4     0.001905
 5     0.000418
Name: children, dtype: float64
 0     0.661914
 1     0.218491
 2     0.093836
 3     0.016559
 20    0.004140
 4     0.003220
-1     0.001380
 5     0.000460
Name: children, dtype: float64

Comparação da coluna education_id entre a tabela df e df_nan:
1    0.707689
0    0.244367
2    0.034564
3    0.013101
4    0.000279
Name: education_id, dtype: float64
1    0.708372
0    0.250230
2    0.031739
3    0.009660
Name: education_id, dtype: float64

Comparação da coluna family_status_id entre a tabela df e df_nan:
0    0.575145
1    0.194053
4    0.130685
3    0.055517
2    0.044599
Name: family_status_id, dtype: float64
0    0.568997
1    0.203312
4    0.132475
3    0.051518
2    0.043698
Name: family_status_id, dtype: float64

Comparação da coluna gender entre a tabela df e df_nan:
F      0.661370
M      

**Conclusão intermediária**

Não foi encontrado nenhuma variação relevante em relação a proporção dos valores entre a tabela **`df`** e a tabela **`df_nan`**. Sendo assim, não é possível afirmar a existência de uma característica específica do cliente como motivo para os valores ausentes.

 **Conclusões**

Não foi encontrdo padrão para para os valores ausentes e essas linhas representam 10% da tabela, sendo assim, foi decidido a subistituição dos valores ausentes. Será utilizada a média para substituir os valores das colunas **`days_employed`** e **`total_income`**, após essas colunas serem tratadas.

Dessa forma, as colunas serão tratadas para a identificação e correção de duplicatas, registros diferentes, artefatos incorretos e valores ausentes.

## Transformação de dados



Verificação da coluna **`education`**:

In [10]:
# Vamos ver todos os valores na coluna de educação para verificar se e quais grafias precisarão ser corrigidas
print(df['education'].unique())

df['education'].value_counts()

["bachelor's degree" 'secondary education' 'Secondary Education'
 'SECONDARY EDUCATION' "BACHELOR'S DEGREE" 'some college'
 'primary education' "Bachelor's Degree" 'SOME COLLEGE' 'Some College'
 'PRIMARY EDUCATION' 'Primary Education' 'Graduate Degree'
 'GRADUATE DEGREE' 'graduate degree']


secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

A coluna **`education`** possui valores iguais inseridos de forma diferente, o tamanho da fonte será diminuído para acabar com as duplicatas.

In [11]:
df['education']= df['education'].str.lower()

In [12]:
# Verificando todos os valores na coluna para ter certeza de que os corrigimos
print(df['education'].unique())

df['education'].value_counts()


["bachelor's degree" 'secondary education' 'some college'
 'primary education' 'graduate degree']


secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

Duplicatas da coluna **`education`** foram resolvidas.

Verificação da coluna **`children`**:

In [13]:
print(df['children'].unique())

print(df['children'].value_counts())

df['children'].value_counts(normalize=True)

[ 1  0  3  2 -1  4 20  5]
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


 0     0.657329
 1     0.223833
 2     0.095470
 3     0.015331
 20    0.003531
-1     0.002184
 4     0.001905
 5     0.000418
Name: children, dtype: float64

A coluna **`children`** possui valores claramente incorretos. A quantidade "-1" de filhos foi provavlemente um erro de digitação, deve representar "1". O valor "20" também parece estar incorreto, pois o valor abaixo é "5", sendo provável foi erro de digitação e representa o valor "2". Dessa forma, os valores "-1" e "20" serão substituídos por "1" e "2".

In [14]:
df['children'] = df['children'].replace([-1,20],[1,2])

In [15]:
# Verificar a coluna `children` novamente para ter certeza de que está tudo corrigido
df['children'].value_counts()


0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Valores incorretos da coluna **`children`** foram substituidos.

Verificação da coluna **`days_employed`**:

In [16]:
print(df['days_employed'].describe())
df['days_employed'].median()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64


-1203.369528770489

In [17]:
len(df.query('days_employed < 0 or days_employed > 30000')) / len(df)

0.8990011614401858

Na coluna **`days_employed`** estão valores negativos e alguns valores muito grandes para para a variável. Esses valores representam 90% das linhas. Os valores negativos serão positivados. Os valores mais altos que 30000 serão divididos por 100, pois,  deve ter havido erro de digitação. Com o valor mais alto real sendo 18388.949901 ou 50 anos, acima desse valor há um salto para um tempo irreal de trabalho.

In [18]:
# Positivar os valores negativos
df['days_employed'] = abs(df['days_employed'])


Conferindo os percentis da coluna.

In [19]:
print(df['days_employed'].quantile([0.25,0.5,0.75,0.8,0.825,0.85,1]))

print()
print('Mediana: ',df['days_employed'].median())

0.250       927.009265
0.500      2194.220567
0.750      5537.882441
0.800      8795.549101
0.825    329983.991540
0.850    340088.707327
1.000    401755.400475
Name: days_employed, dtype: float64

Mediana:  2194.220566878695


In [20]:
df[df['days_employed']<30000].sort_values(by='days_employed',ascending=False).head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16335,1,18388.949901,61,secondary education,1,married,0,F,employee,0,29788.629,real estate transactions
4299,0,17615.563266,61,secondary education,1,married,0,F,business,0,19609.719,purchase of the house
7329,0,16593.472817,60,bachelor's degree,0,married,0,F,employee,0,19951.655,going to university
17838,0,16264.699501,59,secondary education,1,married,0,F,employee,0,8198.235,to buy a car
16825,0,16119.687737,64,secondary education,1,married,0,F,employee,0,14644.430,buy residential real estate


In [21]:
df['days_employed']=df['days_employed'].mask(df['days_employed']>30000.000, df['days_employed'] /100 )

In [22]:
# Verifique o resultado - certifique-se de que está corrigido

df['days_employed'].describe()

count    19351.000000
mean      2583.921756
std       2149.032624
min         24.141633
25%        927.009265
50%       2194.220567
75%       3658.750303
max      18388.949901
Name: days_employed, dtype: float64

Os valores negativos foram positivados e os valores incorretamente grandes tiveram seu valor dividido por 100. A valicição mostrou que os dados foram corrigidos.

Verificação da coluna **`dob_years`**:

In [23]:
## Verifique o `dob_years` para valores suspeitos
print(df['dob_years'].describe())

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64


In [24]:
display(len(df[df['dob_years']==0]['dob_years']))

len(df[df['dob_years']==0]['dob_years']) / len(df)

101

0.004692218350754936

 A coluna **`dob_years`** possui 101 valores 0. Um valor incorreto para a idade, sendo assim, serão substituídos pela mediana.

In [25]:
# Resolução dos problemas na coluna `dob_years`
df['dob_years'].mask(df['dob_years']==0, df['dob_years'].median(),inplace=True)

In [26]:
# Verificação do resultado
print(df['dob_years'].describe())

df[df['dob_years']==0]['dob_years'].count()

count    21525.000000
mean        43.490453
std         12.218595
min         19.000000
25%         34.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64


0

As idades com 0 anos foram substituídas pela mediana. Todos os problemas da coluna **`dob_years`** foram corrigidos.

Verificação da coluna **`family_status`**:

In [27]:
# Vamos ver os valores da coluna family_status
print(df['family_status'].unique())

df['family_status'].value_counts()


['married' 'civil partnership' 'widow / widower' 'divorced' 'unmarried']


married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

A coluna **`family_status`** não possui nenhum problema.

Verificação da coluna **`gender`**:

In [28]:
# Vamos ver os valores na coluna
print(df['gender'].unique())

df['gender'].value_counts()

['F' 'M' 'XNA']


F      14236
M       7288
XNA        1
Name: gender, dtype: int64

O valor "XNA" da coluna **`gender`**, por possuir apenas um valor na coluna inteira, foi decidico que a linha será apagada da tabela.

In [29]:
df = df.drop(df[df['gender'] == 'XNA'].index).reset_index(drop=True)

Valor "XNA" deletado da tabela.

In [30]:
# Verifique o resultado - certifique-se de que está corrigido
df['gender'].value_counts()


F    14236
M     7288
Name: gender, dtype: int64

Verificação da coluna **`income_type`**:

In [31]:
# Vamos ver os valores na coluna
print(df['income_type'].unique())

print(df['income_type'].value_counts())

df['income_type'].value_counts(normalize=True)

['employee' 'retiree' 'business' 'civil servant' 'unemployed'
 'entrepreneur' 'student' 'paternity / maternity leave']
employee                       11119
business                        5084
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64


employee                       0.516586
business                       0.236201
retiree                        0.179149
civil servant                  0.067785
unemployed                     0.000093
entrepreneur                   0.000093
student                        0.000046
paternity / maternity leave    0.000046
Name: income_type, dtype: float64

A coluna **`income_type`** não possui valores problemáticos.

Verificação de valores **duplicados**:

In [32]:
# Verificar duplicatas
df.duplicated().sum()

72

In [33]:
# Aborde as duplicatas, se existirem
df=df.drop_duplicates().reset_index(drop = True)

In [34]:
# Última verificação se temos duplicatas
df.duplicated().sum()

0

In [35]:
#Verifique o tamanho do conjunto de dados que você tem agora após suas primeiras manipulações com ele
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21452 entries, 0 to 21451
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21452 non-null  int64  
 1   days_employed     19350 non-null  float64
 2   dob_years         21452 non-null  int64  
 3   education         21452 non-null  object 
 4   education_id      21452 non-null  int64  
 5   family_status     21452 non-null  object 
 6   family_status_id  21452 non-null  int64  
 7   gender            21452 non-null  object 
 8   income_type       21452 non-null  object 
 9   debt              21452 non-null  int64  
 10  total_income      19350 non-null  float64
 11  purpose           21452 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [36]:
73 / 21525 

0.003391405342624855

O dataframe foi reduzido de 21525 para 21452, foram retiradas 73 linhas com dados duplicados ou incorretos, uma variação de 0,3%. 

## Trabalhando com valores ausentes

As colunas **`total_income`**, **`days_employed`** e **`year`** possuem valores ausentes. Sendo assim, esse dados serão substituídos pelas médias ou medianas, dependendo. 

In [37]:
# Encontre os dicionários
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,3402.660720,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


### Restaurar valores ausentes em `total_income`

Os valores ausentes da coluna **`total_income`** serão abordados primeiro. Os valores ausentes serão relacionados as categorias das colunas **`education`**, **`income_type`** e **`dob_years`**. Será visto a quantidades e proporção de valores ausentes em relação as categorias dessas colunas. O objetivo é procurar qual é a melhor coluna para auxiliar na substituição dos valores ausentes.

A coluna **`dob_years`** é numérica, sendo assim, foi decidido pela criação de uma nova coluna **`age_index`** , armazenando as faixas de idade em categorias.

In [38]:
df['total_income'].describe()

count     19350.000000
mean      26787.266688
std       16475.822926
min        3306.762000
25%       16486.515250
50%       23201.873500
75%       32547.910750
max      362496.645000
Name: total_income, dtype: float64

Relação entre **`total_income`** e **`education`**:

In [39]:
display(df[df['total_income'].isna()]['education'].value_counts())
df[df['total_income'].isna()]['education'].value_counts() / df['education'].value_counts()

secondary education    1478
bachelor's degree       534
some college             69
primary education        21
Name: education, dtype: int64

bachelor's degree      0.101714
graduate degree             NaN
primary education      0.074468
secondary education    0.097423
some college           0.092867
Name: education, dtype: float64

Relação entre **`total_income`** e **`income_type`**:

In [40]:
display(df[df['total_income'].isna()]['income_type'].value_counts())

df[df['total_income'].isna()]['income_type'].value_counts() / df['income_type'].value_counts()

employee         1069
business          501
retiree           386
civil servant     145
entrepreneur        1
Name: income_type, dtype: int64

business                       0.098680
civil servant                  0.099520
employee                       0.096454
entrepreneur                   0.500000
paternity / maternity leave         NaN
retiree                        0.100810
student                             NaN
unemployed                          NaN
Name: income_type, dtype: float64

###  Criação da coluna **`age_index`** :

In [41]:
df['dob_years'].describe()

count    21452.000000
mean        43.469933
std         12.213723
min         19.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [42]:
def creat_column_age(df):
    cut_index = pd.cut(df , bins=[18, 29, 39, 49, 59, 69, 76],   labels= ['18-29', '30-39', '40-49','50-59','60-69','70-75'])
    return cut_index

A função para categorizar os valores da coluna **`dob_years`** foi criada.

In [43]:
# Teste se a função funciona
test = creat_column_age(df['dob_years'])
test.value_counts()

30-39    5662
40-49    5454
50-59    4657
18-29    3179
60-69    2331
70-75     169
Name: dob_years, dtype: int64

O teste da função *creat_column_age* foi bem sucedido. Agora será aplicada a nova coluna **`age_index`**.

In [44]:
# Criar coluna nova com base na função
df['age_index'] = creat_column_age(df['dob_years'])


In [45]:
df['age_index'].value_counts(dropna=False)

30-39    5662
40-49    5454
50-59    4657
18-29    3179
60-69    2331
70-75     169
Name: age_index, dtype: int64

In [46]:
# Verificar como os valores na nova coluna
df[['dob_years','age_index']].head()


,dob_years,age_index
0,42,40-49
1,36,30-39
2,33,30-39
3,32,30-39
4,53,50-59


Coluna **`age_index`** criada.

Relação entre **`total_income`** e **`age_index`**:

In [47]:
display(df[df['total_income'].isna()]['age_index'].value_counts())

df[df['total_income'].isna()]['age_index'].value_counts() / df['age_index'].value_counts()

30-39    553
40-49    529
50-59    479
18-29    296
60-69    236
70-75      9
Name: age_index, dtype: int64

30-39    0.097669
40-49    0.096993
50-59    0.102856
18-29    0.093111
60-69    0.101244
70-75    0.053254
Name: age_index, dtype: float64

Essa relação possivel a melhor correlação e distribuição de valores ausentes entre as categorias.

In [48]:
df_age_orig = df.pivot_table(index='age_index',\
                values='total_income',
                aggfunc=['median','mean'])\
                .rename(columns={'total_income':''})

df_age_orig

,median,mean
,,
age_index,,
18-29,22735.9110,25531.501098
30-39,24667.5280,28312.479963
40-49,24755.6960,28491.929026
50-59,22203.0745,25811.700327
60-69,19817.4400,23242.812818
70-75,18751.3240,20125.658331


Para substituir os valores ausentes da coluna **`total_income`** serão utilizado  as medianas dos valores da coluna **`age_index`**, pois ela tem a relação mais uniformes de valores faltantes, e existe logica em quanto mais velho maior o salario. 

Criação da função para substituir os valores ausentes pela mediana:

In [49]:
def fill_by_median(df, groupby_col, fillna_col):
    value = df.groupby(groupby_col)[fillna_col].transform('median')
    df[fillna_col] = df[fillna_col].fillna(value)
    return df

In [50]:
df = fill_by_median(df, 'age_index', 'total_income')

Funcão aplicada a coluna **`total_income`**.

In [51]:
# Verifique se temos algum erro
display(df['total_income'].isna().value_counts())
display(df['total_income'].describe())

False    21452
Name: total_income, dtype: int64

count     21452.000000
mean      26444.214836
std       15691.083429
min        3306.762000
25%       17217.441750
50%       23238.964500
75%       31328.693750
max      362496.645000
Name: total_income, dtype: float64

Os valores ausentes da coluna **`total_income`** foram substítuidos.

In [52]:
# Verificar o número de entradas nas colunas
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21452 entries, 0 to 21451
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   children          21452 non-null  int64   
 1   days_employed     19350 non-null  float64 
 2   dob_years         21452 non-null  int64   
 3   education         21452 non-null  object  
 4   education_id      21452 non-null  int64   
 5   family_status     21452 non-null  object  
 6   family_status_id  21452 non-null  int64   
 7   gender            21452 non-null  object  
 8   income_type       21452 non-null  object  
 9   debt              21452 non-null  int64   
 10  total_income      21452 non-null  float64 
 11  purpose           21452 non-null  object  
 12  age_index         21452 non-null  category
dtypes: category(1), float64(2), int64(5), object(5)
memory usage: 2.0+ MB


###  Restaurar valores em `days_employed`

Agora os valores ausentes da coluna **`days_employed`** serão abordados. Os valores ausentes serão relacionados as categorias das colunas **`education`**, **`family_status`** e **`income_type`**. Será visto a quantidades e proporção de valores ausentes em relação as categorias dessas colunas. O objetivo é procurar qual é a melhor coluna para auxiliar na substituição dos valores ausentes.

In [53]:
df['days_employed'].describe()

count    19350.000000
mean      2583.933400
std       2149.087546
min         24.141633
25%        926.990457
50%       2194.218768
75%       3658.788094
max      18388.949901
Name: days_employed, dtype: float64

Relação entre **`days_employed`** e **`education`**:

In [54]:
display(df[df['days_employed'].isna()]['education'].value_counts())

df[df['days_employed'].isna()]['education'].value_counts() / df['education'].value_counts()

secondary education    1478
bachelor's degree       534
some college             69
primary education        21
Name: education, dtype: int64

bachelor's degree      0.101714
graduate degree             NaN
primary education      0.074468
secondary education    0.097423
some college           0.092867
Name: education, dtype: float64

Relação entre **`days_employed`** e **`family_status`**:

In [55]:
display(df[df['days_employed'].isna()]['family_status'].value_counts())

df[df['days_employed'].isna()]['family_status'].value_counts() / df['family_status'].value_counts()

married              1196
civil partnership     415
unmarried             285
divorced              112
widow / widower        94
Name: family_status, dtype: int64

married              0.096928
civil partnership    0.100024
unmarried            0.101423
divorced             0.093724
widow / widower      0.098019
Name: family_status, dtype: float64

Relação entre **`days_employed`** e **`income_type`**:

In [56]:
display(df[df['days_employed'].isna()]['income_type'].value_counts())

df[df['days_employed'].isna()]['income_type'].value_counts() / df['income_type'].value_counts()

employee         1069
business          501
retiree           386
civil servant     145
entrepreneur        1
Name: income_type, dtype: int64

business                       0.098680
civil servant                  0.099520
employee                       0.096454
entrepreneur                   0.500000
paternity / maternity leave         NaN
retiree                        0.100810
student                             NaN
unemployed                          NaN
Name: income_type, dtype: float64

Média e mediana de **`days_employed`** por **`income_type`**:

In [57]:
display(df.pivot_table(index ='income_type', values= 'days_employed', aggfunc=['mean','median']))

,mean,median
,days_employed,days_employed
income_type,,
business,2111.470404,1546.333214
civil servant,3399.896902,2689.368353
employee,2326.499216,1574.202821
entrepreneur,520.848083,520.848083
paternity / maternity leave,3296.759962,3296.759962
retiree,3650.034912,3652.133063
student,578.751554,578.751554
unemployed,3664.136527,3664.136527


O parâmetro para substituir os valores ausentes na coluna **`days_employed`** será o **`income_type`**. Por possuir uma distribuição significativa em relação aos dados faltantes.

A mediana será usada por possuir menor variância em relação a *business*, *civil servant*, *employee* da coluna **`income_type`**.

In [58]:
# Vamos escrever uma função que calcule médias ou medianas (dependendo da sua decisão) com base no seu parâmetro identificado

df =  fill_by_median(df, 'income_type', 'days_employed')

In [59]:
# Verifique se a função funciona
display(df['days_employed'].isna().value_counts())
display(df['days_employed'].describe())


False    21452
Name: days_employed, dtype: int64

count    21452.000000
mean      2529.221036
std       2063.920041
min         24.141633
25%       1023.617476
50%       1996.223132
75%       3633.622079
max      18388.949901
Name: days_employed, dtype: float64

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21452 entries, 0 to 21451
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   children          21452 non-null  int64   
 1   days_employed     21452 non-null  float64 
 2   dob_years         21452 non-null  int64   
 3   education         21452 non-null  object  
 4   education_id      21452 non-null  int64   
 5   family_status     21452 non-null  object  
 6   family_status_id  21452 non-null  int64   
 7   gender            21452 non-null  object  
 8   income_type       21452 non-null  object  
 9   debt              21452 non-null  int64   
 10  total_income      21452 non-null  float64 
 11  purpose           21452 non-null  object  
 12  age_index         21452 non-null  category
dtypes: category(1), float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Categorização de dados

Para responder as hipóteses é necessário agrupar os valores das colunas **`purpose`** e **`total_income`** usando a função *.pivot_table()*. Para agrupar e retirar informações relevantes é necessário categorizar os valores dessas colunas.

- Os valores da coluna **`purpose`** serão categorizados pelo tipo de motivo para o empréstimo.
- Os valores da coluna **`total_income`** serão categorizados por nível de renda.

In [61]:
# Exiba os valores dos dados selecionados para categorização
df['purpose'].value_counts()

wedding ceremony                            791
having a wedding                            767
to have a wedding                           765
real estate transactions                    675
buy commercial real estate                  661
housing transactions                        652
buying property for renting out             651
transactions with commercial real estate    650
purchase of the house                       646
housing                                     646
purchase of the house for my family         638
construction of own property                635
property                                    633
transactions with my real estate            627
building a real estate                      624
buy real estate                             620
purchase of my own house                    620
building a property                         619
housing renovation                          607
buy residential real estate                 606
buying my own car                       

In [62]:
# Verifique os valores exclusivos
df['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

Pelas características únicas identificadas nos valores da coluna **`purpose`** foi decidido dividir em quatro categorias que agrupem esses valores. Sendo eles:

- carro: empréstimo para a compra de um carro.
- casa:  empréstimo para a compra de uma casa
- casamento: empréstimo para a celebração de um casamento
- educação:  empréstimo para o pagamento de uma educação superior.

In [63]:
# Vamos escrever uma função para categorizar os dados com base em tópicos comunss
def grup_purpose(data):
    if 'car' in data['purpose']:
        return 'carro'
    if 'hous' in data['purpose'] or 'prop' in data['purpose'] or 'real est' in data['purpose']:
        return 'casa'
    if 'wedd' in data['purpose']:
        return 'casamento'
    if 'educ' in data['purpose'] or 'uni' in data['purpose']:
        return 'educação'

In [64]:
# Crie uma coluna com as categorias e conte os valores para elas
df['categ_purpose'] = df.apply(grup_purpose, axis=1)


In [65]:
df['categ_purpose'].value_counts()

casa         10810
carro         4306
educação      4013
casamento     2323
Name: categ_purpose, dtype: int64

Os dados da coluna **`total_income`** serão categorizados por faixa de renda.

- pobre: grupo de clientes com as 10% menores rendas. Percentil inferior a 0.10.
- classe média: grupo de clientes no intervalo de renda entre os quartis 0.10 e 0.75.
- classe média alta: grupo de clientes no intervalo de renda entre os quartis 0.75 e 0.99.
- rico: clientes que estão no grupo de 1% renda mais elevadas. Quartil superior a 0.99.

In [66]:
# Obter estatísticas resumidas para a coluna
df['total_income'].describe()

count     21452.000000
mean      26444.214836
std       15691.083429
min        3306.762000
25%       17217.441750
50%       23238.964500
75%       31328.693750
max      362496.645000
Name: total_income, dtype: float64

In [67]:
df['total_income'].quantile([0.1,0.75,0.99])

0.10    12594.91110
0.75    31328.69375
0.99    80883.31671
Name: total_income, dtype: float64

In [68]:
# Criar função para categorização em diferentes grupos numéricos com base em intervalos
rico = df['total_income'].quantile(0.99)
media = df['total_income'].quantile(0.75)
pobre = df['total_income'].quantile(0.10)

def grup_income(renda):
    if renda<= pobre:
        return 'pobre'
    if (renda> pobre) and (renda <= media):
        return 'classe média'
    if (renda> media) and (renda < rico):
        return 'classe média alta'
    if (renda >= rico):
        return 'rico'


In [69]:
# Criar coluna com categorias
df['categ_income'] = df['total_income'].apply(grup_income)

In [70]:
# Conte os valores de cada categoria para ver a distribuição
df['categ_income'].value_counts()

classe média         13943
classe média alta     5148
pobre                 2146
rico                   215
Name: categ_income, dtype: int64

In [71]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_index,categ_purpose,categ_income
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49,casa,classe média alta
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39,carro,classe média
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39,casa,classe média
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39,educação,classe média alta
4,0,3402.660720,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-59,casamento,classe média


## Verificar as Hipóteses

**Existe uma correlação entre a quantidade de filhos e do pagamento em dia?**

In [72]:
# Verifique os dados das crianças e do pagamento em dia
df_child_debt =  df.pivot_table(index='children',columns='debt',values='age_index',aggfunc='count')

# Calcular a taxa de inadimplência com base no número de filhos
df_child_debt['porcentagem_de_default'] = 100* (df.groupby('children')['debt'].mean())

display(df_child_debt)

display(df_child_debt.loc[[0,1,2]].sort_values(by='porcentagem_de_default'))

debt,0,1,porcentagem_de_default
children,,,
0,13026.0,1063.0,7.544893
1,4410.0,445.0,9.165808
2,1926.0,202.0,9.492481
3,303.0,27.0,8.181818
4,37.0,4.0,9.756098
5,9.0,NaN,0.000000


debt,0,1,porcentagem_de_default
children,,,
0,13026.0,1063.0,7.544893
1,4410.0,445.0,9.165808
2,1926.0,202.0,9.492481


**Conclusão**

Foi possível identificar uma correlação positiva entre a coluna **`children`** e a **`porcentagem_de_default`**, essa correlação só é vista quando utilizados os valores de  **`children`** com observações suficientes. 

Sendo assim, pode-se afirmar que quantos mais filhos menos pagamentos em dia, porém, a variação relevante é entre 0 filhos e 1 filho( de 7,5% para 9.2%), após isso, o crescimento já não é relevante. Logo, deve-se tratar que ter filhos ou não afeta o pagamento do empréstimo.

**Existe uma correlação entre o status familiar e o pagamento em dia?**

In [73]:
# Verifique os dados de status da família e do pagamento em dia
df_fam_debt =  df.pivot_table(index='family_status',columns='debt',values='age_index',aggfunc='count')

# Calcular a taxa padrão com base no status da família
df_fam_debt['porcentagem_de_default'] = 100* (df.groupby('family_status')['debt'].mean())

display(df_fam_debt.sort_values(by='porcentagem_de_default'))

debt,0,1,porcentagem_de_default
family_status,,,
widow / widower,896,63,6.569343
divorced,1110,85,7.112971
married,11408,931,7.545182
civil partnership,3761,388,9.351651
unmarried,2536,274,9.750890


**Conclusão**

Foi possível identificar uma correlação entre **`debt`** e **`porcentagem_de_default`**. Com os indivíduos solteiros(unmarried) com a maior porcentagem de não pagamento(9,7%) e os indivíduos viúvos(6,5%) com a menor.

Dessa forma, percebe-se que os indivíduos que nunca foram casados possuem uma propensão maior a inadimplência, entre 9,3 a 9,7%. Os indivíduos que já foram casados possuem uma tendência menor, entre 6,5% e 7,5%.

**Existe uma correlação entre o nível de renda e o pagamento em dia?**

In [74]:
# Verifique os dados do nível de renda e do pagamento em dia
df_inco_debt =  df.pivot_table(index='categ_income',columns='debt',values='age_index',aggfunc='count')


# Calcular a taxa de inadimplência com base no nível de renda
df_inco_debt['porcentagem_de_default'] = 100* (df.groupby('categ_income')['debt'].mean())

display(df_inco_debt.sort_values(by='porcentagem_de_default'))


debt,0,1,porcentagem_de_default
categ_income,,,
rico,201,14,6.511628
classe média alta,4779,369,7.167832
pobre,1989,157,7.315937
classe média,12742,1201,8.613641


**Conclusão**

Pelo uso das categorias da coluna **`categ_income`** em relação a **`debt`**. Percebe-se que a diferença de inadimplência entre as categorias não possui uma variação muito relevante entre elas, com as *categorias classe média alta* e *pobre* tendo o mesmo nível de inadimplência. 

A categoria *rico* possui uma taxa de inadimplência menor em relação as outras mas possui poucas observações, sendo assim, não é possível dizer que a diferença é significativa.

**Como a finalidade do crédito afeta a taxa de inadimplência?**

In [75]:
# Confira os percentuais de inadimplência para cada finalidade de crédito e analise-o3333s
df_por_debt =  df.pivot_table(index='categ_purpose',columns='debt',values='age_index',aggfunc='count')


# Calcular a taxa de inadimplência com base no propósito
df_por_debt['porcentagem_de_default'] = 100* (df.groupby('categ_purpose')['debt'].mean())

display(df_por_debt.sort_values(by='porcentagem_de_default'))

debt,0,1,porcentagem_de_default
categ_purpose,,,
casa,10028,782,7.234043
casamento,2137,186,8.006888
educação,3643,370,9.220035
carro,3903,403,9.359034


**Conclusão**

Pelo uso das categorias da coluna **`categ_purpose`** em relação a **`debt`**. Percebe-se que os empréstimos com finalidade imobiliária possuem a menor taxa de inadimplência, de 7,2%. Os empréstimos para aquisição de educação superior e carro possuem maiores taxas de inadimplência, de 9,22% e 9,35%. A taxa de inadimplência para financiar um casamento é de 8%.

Dessa forma, os empréstimos para o mercado imobiliario pode ser considerado o mais seguro e o para carros o menos seguros. Com as informações disponíveis não é possível saber o motivo.

## Conclusão Geral 

Inicialmento houve a exploração dos dados, com o objetivo de identificar dados incorretos, ausentes ou disprepantes. Foram identifacados que as colunas **`days_employed`**, **`education`** e **`total_income`** precisavam de correção.

Em relação aos valores ausentes, 2174 observações possuiam valores ausentes representando 10% do dataframe. Não foi identificada uma razão para os valores ausentes. Foi decidido preencher os valores ausentes. Para a correção dos valores negativos foi usado o módulo. Para os valores discrepantes, foram divididos para ficarem na ordem de grandeza dos outros valores. Em relação aos dados duplicados ou incorretos, foram 73 linhas, uma variação de 0,3%.

Foi criado a coluna **`age_index`** para categorizar os dados da coluna **`dob_years`**. Para categorizar a coluna **`total_income`** foi criada a coluna **`categ_income`** e para a coluna **`purpose`** foi criado a coluna **`categ_income`**.


As conclusões em relação as perguntas foram :

Existe uma correlação entre a quantidade de filhos e do pagamento em dia?
- Existe correlação entre ter filhos ou não e o pagamento em dia. O números de filhos não é relevante a taxa de inadimplência


Existe uma correlação entre o status familiar e o pagamento em dia?
- Os indivíduos que nunca foram casados possuem uma propensão maior a inadimplência, entre 9,3 a 9,7%. Os indivíduos que já foram casados possuem uma tendência menor, entre 6,5% e 7,5%.

Existe uma correlação entre o nível de renda e o pagamento em dia?
- O nível de renda não possui uma variação da taxa de inadimplência muito relevante entre elas, com as categorias classe média alta e pobre tendo o mesmo nível de inadimplência.

Como a finalidade do crédito afeta a taxa de inadimplência?
- Os empréstimos com finalidade imobiliária possuem a menor taxa de inadimplência, seguido pelo casamento. Os empréstimos para aquisição de educação superior e carro possuem as maiores taxas de inadimplências.